<a href="https://colab.research.google.com/github/garg-chhavi/CVND_Exercises/blob/master/zerogans_protein_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Human Protein Multi Label Image Classification 

This is a starter notebook for the competition [Zero to GANs - Human Protein Classification](https://www.kaggle.com/c/jovian-pytorch-z2g)

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import f1_score
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid
%matplotlib inline

## Exploring the Data

When you create a notebook with the "Notebooks" tab of a Kaggle competition, the data is automatically included in the `../input` folder. You can explore the files in the sidebar. Let us create some constants acess the data directories and CSV files.

In [ ]:
DATA_DIR = '../input/jovian-pytorch-z2g/Human protein atlas'

TRAIN_DIR = DATA_DIR + '/train'                           # Contains training images
TEST_DIR = DATA_DIR + '/test'                             # Contains test images

TRAIN_CSV = DATA_DIR + '/train.csv'                       # Contains real labels for training images
TEST_CSV = '../input/jovian-pytorch-z2g/submission.csv'   # Contains dummy labels for test image

The `train.csv` file contains image IDs and labels for training data. Note that this is a multi-label classification problem. Each image can have more than one type of protein.

In [ ]:
!head "{TRAIN_CSV}"

Similarly, `submission.csv` contains image IDs for test data. However, since the goal of this competition is to make predictions for the test set, the `submission.csv` files contains **dummy labels**, which you need to replace with your predictions and sumbit to the competition on the "Submission" tab. In other words, we don't have the labels for the test set.

In [ ]:
!head "{TEST_CSV}"

The image files are named `<image-id>.png` and can be found in the respective `train` and `test` folders.

In [ ]:
!ls "{TRAIN_DIR}" | head

Let's load the `train.csv` file into a Pandas dataframe

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

In [ ]:
#train_df.iloc[1,[1]]

for l in str(train_df.iloc[1,[1]][0]).split(' '):
    print(l)
    
train_df.iloc[1,[1]][0]
str(train_df.iloc[1,[1]][0])

Let's also put the textual labels in a dictionary for later use.

In [ ]:
labels = {
    0: 'Mitochondria',
    1: 'Nuclear bodies',
    2: 'Nucleoli',
    3: 'Golgi apparatus',
    4: 'Nucleoplasm',
    5: 'Nucleoli fibrillar center',
    6: 'Cytosol',
    7: 'Plasma membrane',
    8: 'Centrosome',
    9: 'Nuclear speckles'
}

To create a tensor from the labels, we will encode the labels as vectors of 1s & 0s. For example, if the labels are `'2 4 5'`, the correspoding vector for it would be `[0, 0, 1, 0, 1, 1, 0, 0, 0, 0]`. Let's define helper funtions to encode labels into tensors and vice versa.

In [ ]:
def encode_label(label):
    target = torch.zeros(10)
    for l in str(label).split(' '):
        target[int(l)] = 1.
    return target

def decode_target(target, text_labels=False, threshold=0.5):
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
            if text_labels:
                result.append(labels[i] + "(" + str(i) + ")")
            else:
                result.append(str(i))

    return ' '.join(result)
    

In [ ]:
all_targets = []
for row in range(train_df.shape[0]):
    all_targets.append(encode_label(str(train_df.iloc[row,[1]][0])).tolist())
all_targets = pd.DataFrame(all_targets, columns= labels.values())
all_targets.apply(sum).plot.bar()

In [ ]:
encode_label('2 4 5')

In [ ]:
decode_target(torch.tensor([0, 0, 1, 0, 1, 1, 0, 0, 0, 0.]))

In [ ]:
decode_target(torch.tensor([0, 0, 1, 0, 1, 1, 0, 0, 0, 0.]), text_labels=True)

## Creating Datasets & Data Loaders

We can now create a custom dataset by extending the `Dataset` class from PyTorch. We need to define the `__len__` and `__getitem__` methods to create a dataset. We'll also provide the option of adding transforms into the constructor.

In [ ]:
class HumanProteinDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_id, img_label = row['Image'], row['Label']
        img_fname = self.root_dir + "/" + str(img_id) + ".png"
        img = Image.open(img_fname)
        #img = img1.filter(ImageFilter.BLUR)
        #print(type(img))
        if self.transform:
            img = self.transform(img)
            img = img
        return img, encode_label(img_label)

Transforms can be chained using `transforms.Compose`. For instance, you may add `transforms.Resize(128)` before `transforms.ToTensor()` to resize images to size 128x128 before converting them into tensors. See the full list of transforms here: https://pytorch.org/docs/master/torchvision/transforms.html

In [ ]:
TRAIN_DIR

In [ ]:
transform = transforms.Compose([transforms.RandomCrop(400),
                                transforms.RandomRotation((45,90)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor() ])
dataset = HumanProteinDataset(TRAIN_CSV, TRAIN_DIR, transform=transform)

In [ ]:
dataset[0][0].min(), dataset[0][0].max(), dataset[0][0].mean(), dataset[0][0].std()

In [ ]:
# Normalizing does not restrict limit between -1 and 1
a = ((dataset[0][0] - 0.9416)/(.0898))
a.min(), a.max(), a.mean(), a.std()

Let's check how many samples the dataset contains

In [ ]:
len(dataset)

In [ ]:
dataset[0][1].shape, dataset[0][0].shape

Let's take a look at a sample image from the dataset. We'll define a function `show_sample` to help us. We will also include the option to invert the image before showing it, because the original images are quite dark.

In [ ]:
def show_sample(img, target, invert=True):
    if invert:
        plt.imshow(1 - img.permute((1, 2, 0)))
    else:
        plt.imshow(img.permute(1, 2, 0))
    print('Labels:', decode_target(target, text_labels=True))
    

Here's a sample image without the colors inverted.

In [ ]:
dataset[0][1]

In [ ]:
show_sample(dataset[2][0],dataset[2][1], invert=False)

Here's the same image viewed with the colors inverted

In [ ]:
show_sample(*dataset[2])


### Training & Validation sets

As a good practice, we should split the data into training and validation datasets. Let's fix a seed for PyTorch (to ensure we always get the same validation set), and create the datasets using `random_split`.

In [ ]:
torch.manual_seed(10)

I'm using a validation percentage of 10%, but you can use a smaller or larger percentage. One good strategy is to determine a good set of hyperparameters, and then retrain on a smaller validation set for your final submission.

In [ ]:
val_pct = 0.1
val_size = int(val_pct * len(dataset))
train_size = len(dataset) - val_size

In [ ]:
train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

### Data Loaders

In [ ]:
batch_size = 64

In [ ]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)

In [ ]:
def show_batch(dl, invert=True):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(16, 8))
        ax.set_xticks([]); ax.set_yticks([])
        data = 1-images if invert else images
        ax.imshow(make_grid(data, nrow=16).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl)

## View sample images with lables

In [ ]:
dataiter = iter(train_dl)
img = dataiter.next()
images = img[0].numpy()

labels_print = img[1]
fig = plt.figure(figsize=(40, 40))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(8, batch_size/8, idx+1, xticks=[], yticks=[])
    a = 1 - (images[idx].transpose((1,2,0)))
    ax.imshow(a)
    ax.set_title(decode_target(labels_print[idx], text_labels=True))

In [ ]:
dataiter = iter(train_dl)
img = dataiter.next()
images = img[0].numpy()

labels_print = img[1]
fig = plt.figure(figsize=(40, 40))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(8, batch_size/8, idx+1, xticks=[], yticks=[])
    a = (images[idx].transpose((1,2,0)))
    ax.imshow(a)
    ax.set_title(decode_target(labels_print[idx], text_labels=True))

In [ ]:
#show_batch(train_dl, invert = False)

## Model

In [ ]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [ ]:
class MultilabelImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, targets = batch 
        out = self(images)                      
        loss = F.binary_cross_entropy(out, targets)      
        return loss, out, targets
    
    def validation_step(self, batch):
        images, targets = batch 
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, train_f_score: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['train_loss'], result['train_f_score'], result['val_loss'], result['val_score']))

In [ ]:
class ProteinCnnModel2(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=True)
        #self.network = models.vgg16(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        #for layers in self.network.parameters():
         #   layers.requires_grad = False
        
        #for layers in (self.network.layer4[2].parameters()):
        #    layers.requires_grad = True
        
        #self.network.fc = nn.Sequential(    
         #                               nn.Linear(num_ftrs, 128),
          #                              #nn.ReLU(),
           #                             #nn.Dropout(0.5),
            #                            #nn.Linear(256, 128),
             #                           nn.ReLU(),
              #                          nn.Dropout(0.4),
               #                         nn.Linear(128, 10))
        self.network.fc = nn.Linear(num_ftrs, 10)
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))

In [ ]:
model = ProteinCnnModel2()
model

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

In [ ]:
def try_batch(dl):
    for images, labels in dl:
        print('images.shape:', images.shape)
        out = model(images)
        print('out.shape:', out.shape)
        print('out[0]:', out[0])
        break

try_batch(train_dl)

If your kernel runs out of memory here, you might need to reduce your batch size.

## Training the model

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD, best_validation_score = 1):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), lr)
    #best_validation_score = 10
    best_model = model
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        train_fscore = []
        for batch in tqdm(train_loader):
            loss, train_out, train_targets = model.training_step(batch)
            batch_train_score = F_score(train_out, train_targets)
            train_fscore.append(batch_train_score)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['train_f_score'] = torch.stack(train_fscore).mean().item()
        model.epoch_end(epoch, result)
        
        if(result['val_score'] > best_validation_score):
            best_validation_score = result['val_score']
            best_model = model
            torch.save(model.state_dict(), 'resnet_1.pth')
            
        history.append(result)
    return history, best_model, best_validation_score

In [ ]:
model = to_device(ProteinCnnModel2(), device)

In [ ]:
evaluate(model, val_dl)

In [ ]:
#os.listdir("../input/dictionary")
#os.getcwd()

In [ ]:
checkpoint = torch.load("../input/dictionary/resnet_1.pth")
model.load_state_dict(checkpoint)

In [ ]:
for layer in model.network.parameters():
    print(layer)

In [ ]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 1e-2

In [ ]:
## %%time
best_validation_score = 0.6238097548484802
history, model, best_validation_score = fit(num_epochs, lr, model, train_dl, val_dl, opt_func, best_validation_score)

In [ ]:
best_validation_score

In [ ]:
%%time
history, model, best_validation_score =  fit(5, lr, model, train_dl, val_dl, opt_func, best_validation_score)

In [ ]:
#torch.save(model.state_dict(), 'resnet_1.pth')

In [ ]:
%%time
history, model, best_validation_score  = fit(5, .0001, model, train_dl, val_dl, opt_func, best_validation_score)

In [ ]:
2

In [ ]:
history

In [ ]:
#checkpoint = torch.load(PATH)

In [ ]:
val_score_list = [x['val_score'] for x in history]
plt.plot(val_score_list)
val_loss_list = [x['val_loss'] for x in history]
plt.plot(val_loss_list)

In [ ]:
#torch.save(history, 'resnet_1_history.pth')

## Making predictions & submission

To start with, let's create a helper function to make a prediction on a single image.

In [ ]:
def predict_single(image):
    xb = image.unsqueeze(0)
    xb = to_device(xb, device)
    preds = model(xb)
    prediction = preds[0]
    print("Prediction: ", prediction)
    show_sample(image, prediction)

Next, let's create a test dataset using the `submission.csv` file. Note that the file contains dummy labels (always `'0'`). We'll use the same transforms we used for the training set.

In [ ]:
test_dataset = HumanProteinDataset(TEST_CSV, TEST_DIR, transform=transform)

In [ ]:
img, target = test_dataset[0]
img.shape

Let's try predicting the labels for some sample images

In [ ]:
predict_single(test_dataset[100][0])

In [ ]:
predict_single(test_dataset[74][0])

## Creating a submission file

In [ ]:
test_dl = DeviceDataLoader(DataLoader(test_dataset, batch_size, num_workers=2, pin_memory=True), device)

In [ ]:
@torch.no_grad()
def predict_dl(dl, model):
    torch.cuda.empty_cache()
    batch_probs = []
    for xb, _ in tqdm(dl):
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    return [decode_target(x) for x in batch_probs]

In [ ]:
test_preds = predict_dl(test_dl, model)

In [ ]:
best_validation_score

Let us know create a submission file with these predictions

In [ ]:
submission_df = pd.read_csv(TEST_CSV)
submission_df.Label = test_preds
submission_df.head()

We can now save it batck to CSV, and download the file from the sidebar (check the output folder)

In [ ]:
sub_fname = 'resnet34_submission_1.csv'

In [ ]:
submission_df.to_csv(sub_fname, index=False)

You can now upload this submission file here: https://www.kaggle.com/c/jovian-pytorch-z2g/submit

## Save to Jovian

In [ ]:
!pip install jovian --upgrade

     |████████████████████████████████| 94 kB 1.8 MB/s eta 0:00:011
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6500 sha256=1e397a6c7ebe159968b26bfccbb987f539cb668ccb4b6969be57a149e655ea5b
  Stored in directory: /root/.cache/pip/wheels/2a/ea/87/dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid


In [ ]:
import jovian

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit(project='zerogans-protein-competition')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Detected Kaggle notebook...
[jovian] Please enter your API key ( from https://jovian.ml/ ):
API KEY: ········
[jovian] Uploading notebook to https://jovian.ml/garg-chhavi/zerogans-protein-competition


<IPython.core.display.Javascript object>

You can also use the "Save Version" button on Kaggle itself, to save a copy on your Kaggle profile.